In [397]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime
import math
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [398]:
# past_data = pd.read_csv('2014-2019.csv')
# past_data = past_data[:1459]
# tmp_data = pd.read_csv('2018-2020.csv')
# past_data = past_data.append(tmp_data)
# current_data = pd.read_csv('2021year.csv')
# data = past_data.append(current_data)
# data = data.reset_index()
# data['備轉'] = data['備轉容量(萬瓩)']*10
# # tmp = []
# # for i in data.index:
# #     tmp.append(datetime.datetime.strptime(str(data.loc[i, '日期']), '/%Y/%m/%d'))
# # data['date'] = [i.weekday() for i in tmp]
# def normalize(train):
#     train_norm = train.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
#     return train_norm
# data[['backup']] = normalize(data[['備轉']])
data = pd.read_csv('data.csv')

data.to_csv("data.csv")

In [272]:
 def create_data(past = 7, future = 7):
    # model = build()
    x_train = []
    y_train = []
    x_test = []
    y_test = []
    for i in range(past+future, len(data)+1):
        t = data.iloc[i-past-future:i-future][['備轉']]
        y = data['備轉'].iloc[i-future:i]
        if i < 2500:
            x_train.append(t)
            y_train.append(y)
        else:
            x_test.append(t)
            y_test.append(y)

    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_test = np.array(x_test)
    y_test = np.array(y_test)
    return x_train, y_train, x_test, y_test

# x_train = np.reshape(x_train, (1, x_train.shape[0], x_train.shape[1]))
# y_train = np.reshape(y_train, (1, x_train.shape[0], x_train.shape[1]))

# x_test = np.reshape(x_test, (1, x_test.shape[0], x_test.shape[1]))
# y_test = np.reshape(y_test, (1, x_test.shape[0], x_test.shape[1]))

# # print(y_test[-1])
x_train, y_train, x_test, y_test = create_data(7, 1)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
# print(y_test[-1])
# print(x_test.shape)

(2492, 7, 1)
(2492, 1)
(136, 7, 1)
(136, 1)


In [23]:
#mean with before days
def mean(max_len):
    err = 0
    predict = 0
    count = []
    outputs = data['備轉'][0]
    for i in data['備轉']:
        if len(count) > max_len:
            count.pop(0)
        if len(count) > 0:
            outputs = i
            predict = sum(count)/len(count)
            err += (outputs - predict)**2
        count.append(outputs)
    return math.sqrt(err/len(data))

        
def predict_mean(max_len):
    count = data['備轉'][:max_len]
    count = count.values.tolist()
    predict_list = []
    for i in range(7):
        predict = sum(count) / max_len
        predict_list.append(predict)
        count = np.de
        count.append(predict)
    return predict_list
for i in range(15):
    print(mean(i))

0.0
642.9340703168365
627.0390414613433
616.1930709752745
617.5551408259772
621.623426084799
620.6316440890741
618.6411731370952
621.2584252656378
625.4142454154462
629.5416182583996
633.5350609310615
636.8481325524093
637.8723200468373
636.0526255786066


In [119]:
def mean_model(x_train, y_train, x_test, y_test, max_past_len, max_future_len, mode):
    #test mode
    if mode == "test":
        count = data['備轉'][:max_past_len]
        count = count.values.tolist()
        predict_list = []
        for i in range(7):
            predict = sum(count) / max_past_len
            predict_list.append(predict)
            count = count.pop(0)
            count.append(predict)
        return predict_list
    #train mode
    else:
        err = 0
        result = []
        for i, j in zip(x_test, y_test):
            count = [k[0] for k in i]
            for h in j:
                predict = sum(count)/len(count)
                err += (predict-h)**2
                count.pop(0)
                count.append(predict)
            err = math.sqrt(err/max_future_len)
            result.append(err)
            err = 0
            
        return sum(result)/len(result)
for i in range(1, 14):
    for j in range(1, 8):
        x_train, y_train, x_test, y_test = create_data(i, j)

        print("past",i, "future", j, mean_model(x_train, y_train, x_test, y_test, i, j, "train"))


past 1 future 1 147.17857142857142
past 1 future 2 174.88777477607283
past 1 future 3 192.2024142435736
past 1 future 4 200.61268519473688
past 1 future 5 205.4574970999419
past 1 future 6 204.46915191086103
past 1 future 7 208.64931358349943
past 2 future 1 149.39285714285714
past 2 future 2 172.22694388446624
past 2 future 3 185.78040835445321
past 2 future 4 192.85902368500973
past 2 future 5 194.56401497240628
past 2 future 6 193.01547473302065
past 2 future 7 194.44542292522985
past 3 future 1 138.9642857142857
past 3 future 2 170.36959659740302
past 3 future 3 181.6101492461706
past 3 future 4 184.42607349554638
past 3 future 5 185.57449668031924
past 3 future 6 184.79461222622655
past 3 future 7 184.42008064513536
past 4 future 1 141.99107142857142
past 4 future 2 172.37699226483738
past 4 future 3 180.44131475638298
past 4 future 4 183.11675251083264
past 4 future 5 182.42475258807352
past 4 future 6 180.3342507695543
past 4 future 7 179.06038490334777
past 5 future 1 144.64285

In [107]:
def mean_model(x_train, y_train, x_test, y_test, max_len, mode):
    #test mode
    if mode == "test":
        count = data['備轉'][:max_len]
        count = count.values.tolist()
        predict_list = []
        for i in range(7):
            predict = sum(count) / max_len
            predict_list.append(predict)
            count = count.pop(0)
            count.append(predict)
        return predict_list
    #train mode
    else:
        err = 0
        for i,j in zip(x_test, y_test):
            i = i[0]
            predict = i.mean()
            err += (predict-j)**2
        return math.sqrt(err/len(x_test))
    for i in range(1, 11):
        x_train, y_train, x_test, y_test = create_data(i, 1)
        print(i, mean_model(x_train, y_train, x_test, y_test, i, "train"))

In [129]:
 def create_nor_data(past = 7, future = 7):
    # model = build()
    x_train = []
    y_train = []
    x_test = []
    y_test = []
    print(past)
    print(len(data))
    for i in range(past+future, len(data)+1):
        t = data.iloc[i-past-future:i-future][['backup']]
        y = data['備轉'].iloc[i-future:i]
        if i < 2600:
            x_train.append(t)
            y_train.append(y)
        else:
            x_test.append(t)
            y_test.append(y)

    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_test = np.array(x_test)
    y_test = np.array(y_test)
    return x_train, y_train, x_test, y_test

# x_train = np.reshape(x_train, (1, x_train.shape[0], x_train.shape[1]))
# y_train = np.reshape(y_train, (1, x_train.shape[0], x_train.shape[1]))

# x_test = np.reshape(x_test, (1, x_test.shape[0], x_test.shape[1]))
# y_test = np.reshape(y_test, (1, x_test.shape[0], x_test.shape[1]))

# print(y_test[-1])
x_train, y_train, x_test, y_test = create_data(8, 1)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_test[-1].shape)

(2599, 8, 1)
(2599, 1)
(28, 8, 1)
(28, 1)
(8, 1)


In [135]:
def buildmanytomany(shape):
    model = Sequential()
    model.add(LSTM(10, input_length=shape[1], input_dim=shape[2], return_sequences=True))
    model.add(TimeDistributed(Dense(1)))
    model.compile(loss="mse", optimizer="adam")
    model.summary()
    return model


In [ ]:
def build(shape):
    model = Sequential()
    model.add(LSTM(units=50, input_shape=(day, 2), return_sequences=True))
    model.add(LSTM(units=50, return_sequences=True))
#     model.add(Dropout(0.2))
    # output shape: (1, 1)
    model.add(Dropout(0.2))
    model.add(LSTM(units=50,  return_sequences=True))
    # output shape: (1, 1)
    model.add(Dropout(0.2))

    model.add(Dense(units = 1))
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')
    model.summary()

In [145]:
x_train, y_train, x_test, y_test = create_data(7, 7)
regressor = buildmanytomany(x_train.shape)
print(x_train.shape)
# 進行訓練
regressor.fit(x_train, y_train, epochs = 650)



Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 7, 10)             480       
_________________________________________________________________
time_distributed_3 (TimeDist (None, 7, 1)              11        
Total params: 491
Trainable params: 491
Non-trainable params: 0
_________________________________________________________________
(2586, 7, 1)
Epoch 1/650
81/81 [==============================] - 2s 3ms/step - loss: 9926764.1463
Epoch 2/650
81/81 [==============================] - 0s 3ms/step - loss: 9671203.5244
Epoch 3/650
81/81 [==============================] - 0s 3ms/step - loss: 9712187.1463
Epoch 4/650
81/81 [==============================] - 0s 3ms/step - loss: 9742854.4146
Epoch 5/650
81/81 [==============================] - 0s 3ms/step - loss: 9797463.3293
Epoch 6/650
81/81 [==============================] - 0s 4ms/step - loss: 96

81/81 [==============================] - 0s 3ms/step - loss: 9579918.4024
Epoch 89/650
81/81 [==============================] - 0s 3ms/step - loss: 9754824.0854
Epoch 90/650
81/81 [==============================] - 0s 3ms/step - loss: 9876151.4390
Epoch 91/650
81/81 [==============================] - 0s 3ms/step - loss: 9444548.1220
Epoch 92/650
81/81 [==============================] - 0s 3ms/step - loss: 9951631.7561
Epoch 93/650
81/81 [==============================] - 0s 3ms/step - loss: 9470829.1463
Epoch 94/650
81/81 [==============================] - 0s 3ms/step - loss: 9612766.0122
Epoch 95/650
81/81 [==============================] - 0s 3ms/step - loss: 9787644.0122
Epoch 96/650
81/81 [==============================] - 0s 3ms/step - loss: 9616190.3293
Epoch 97/650
81/81 [==============================] - 0s 3ms/step - loss: 9560022.8415
Epoch 98/650
81/81 [==============================] - 0s 3ms/step - loss: 9645957.3293
Epoch 99/650
81/81 [==============================] - 0s

81/81 [==============================] - 0s 3ms/step - loss: 9488415.2195
Epoch 275/650
81/81 [==============================] - 0s 4ms/step - loss: 9213085.7134
Epoch 276/650
81/81 [==============================] - 0s 4ms/step - loss: 9331691.9878
Epoch 277/650
81/81 [==============================] - 0s 4ms/step - loss: 9270762.2439
Epoch 278/650
81/81 [==============================] - 0s 4ms/step - loss: 9180096.9695
Epoch 279/650
81/81 [==============================] - 0s 4ms/step - loss: 9278555.2317
Epoch 280/650
81/81 [==============================] - 0s 3ms/step - loss: 9253529.3598
Epoch 281/650
81/81 [==============================] - 0s 3ms/step - loss: 9267273.9390
Epoch 282/650
81/81 [==============================] - 0s 3ms/step - loss: 9387151.5976
Epoch 283/650
81/81 [==============================] - 0s 4ms/step - loss: 9272126.2195
Epoch 284/650
81/81 [==============================] - 0s 4ms/step - loss: 9128347.4695
Epoch 285/650
81/81 [=========================

81/81 [==============================] - 0s 3ms/step - loss: 8778127.3415
Epoch 461/650
81/81 [==============================] - 0s 3ms/step - loss: 9030104.8537
Epoch 462/650
81/81 [==============================] - 0s 3ms/step - loss: 9116034.7805
Epoch 463/650
81/81 [==============================] - 0s 3ms/step - loss: 8901605.6890
Epoch 464/650
81/81 [==============================] - 0s 3ms/step - loss: 8833604.3841
Epoch 465/650
81/81 [==============================] - 0s 3ms/step - loss: 9050961.0366
Epoch 466/650
81/81 [==============================] - 0s 3ms/step - loss: 9114736.3780
Epoch 467/650
81/81 [==============================] - 0s 3ms/step - loss: 8986064.3110
Epoch 468/650
81/81 [==============================] - 0s 3ms/step - loss: 8924691.1098
Epoch 469/650
81/81 [==============================] - 0s 3ms/step - loss: 9079195.5122
Epoch 470/650
81/81 [==============================] - 0s 3ms/step - loss: 8713715.6341
Epoch 471/650
81/81 [=========================

81/81 [==============================] - 0s 3ms/step - loss: 8619132.8720
Epoch 647/650
81/81 [==============================] - 0s 3ms/step - loss: 8536030.2866
Epoch 648/650
81/81 [==============================] - 0s 3ms/step - loss: 8734249.3293
Epoch 649/650
81/81 [==============================] - 0s 3ms/step - loss: 8628718.9268
Epoch 650/650
81/81 [==============================] - 0s 3ms/step - loss: 8745129.3049


In [191]:
def buildManyToOneModel(shape):
    model = Sequential()
    model.add(LSTM(10, input_length=shape[1], input_dim=shape[2], return_sequences=True))
    model.add(Dense(units = 10))
    model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss="mse", optimizer="adam")
    model.summary()
    return model

In [201]:
x_train, y_train, x_test, y_test = create_data(8, 1)
print(x_train.shape)
regressor = buildManyToOneModel(x_train.shape)
# 進行訓練
regressor.fit(x_train[-100:], y_train[-100:], epochs = 350)



(2599, 8, 1)
Model: "sequential_47"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_64 (LSTM)               (None, 8, 10)             480       
_________________________________________________________________
dense_145 (Dense)            (None, 8, 10)             110       
_________________________________________________________________
dropout_64 (Dropout)         (None, 8, 10)             0         
_________________________________________________________________
dense_146 (Dense)            (None, 8, 10)             110       
_________________________________________________________________
dropout_65 (Dropout)         (None, 8, 10)             0         
_________________________________________________________________
dense_147 (Dense)            (None, 8, 1)              11        
Total params: 711
Trainable params: 711
Non-trainable params: 0
__________________________________________

4/4 [==============================] - 0s 4ms/step - loss: 9179503.2000
Epoch 84/350
4/4 [==============================] - 0s 4ms/step - loss: 9209521.4000
Epoch 85/350
4/4 [==============================] - 0s 3ms/step - loss: 9207296.0000
Epoch 86/350
4/4 [==============================] - 0s 3ms/step - loss: 9152551.4000
Epoch 87/350
4/4 [==============================] - 0s 3ms/step - loss: 9188971.2000
Epoch 88/350
4/4 [==============================] - 0s 3ms/step - loss: 9136272.6000
Epoch 89/350
4/4 [==============================] - 0s 3ms/step - loss: 9119201.4000
Epoch 90/350
4/4 [==============================] - 0s 3ms/step - loss: 9135325.0000
Epoch 91/350
4/4 [==============================] - 0s 3ms/step - loss: 8972389.2000
Epoch 92/350
4/4 [==============================] - 0s 3ms/step - loss: 9015495.4000
Epoch 93/350
4/4 [==============================] - 0s 3ms/step - loss: 8979849.6000
Epoch 94/350
4/4 [==============================] - 0s 3ms/step - loss: 901577

4/4 [==============================] - 0s 3ms/step - loss: 6051621.2000
Epoch 179/350
4/4 [==============================] - 0s 3ms/step - loss: 5943457.8000
Epoch 180/350
4/4 [==============================] - 0s 3ms/step - loss: 5935383.9000
Epoch 181/350
4/4 [==============================] - 0s 3ms/step - loss: 5768137.7000
Epoch 182/350
4/4 [==============================] - 0s 3ms/step - loss: 5788321.9000
Epoch 183/350
4/4 [==============================] - 0s 3ms/step - loss: 5742480.8000
Epoch 184/350
4/4 [==============================] - 0s 3ms/step - loss: 5683150.4000
Epoch 185/350
4/4 [==============================] - 0s 3ms/step - loss: 5697767.3000
Epoch 186/350
4/4 [==============================] - 0s 3ms/step - loss: 5583597.4000
Epoch 187/350
4/4 [==============================] - 0s 3ms/step - loss: 5494740.8000
Epoch 188/350
4/4 [==============================] - 0s 4ms/step - loss: 5407740.8000
Epoch 189/350
4/4 [==============================] - 0s 3ms/step - l

4/4 [==============================] - 0s 3ms/step - loss: 1743619.7750
Epoch 274/350
4/4 [==============================] - 0s 3ms/step - loss: 1670303.1500
Epoch 275/350
4/4 [==============================] - 0s 3ms/step - loss: 1689727.8750
Epoch 276/350
4/4 [==============================] - 0s 3ms/step - loss: 1631817.3500
Epoch 277/350
4/4 [==============================] - 0s 3ms/step - loss: 1624753.0750
Epoch 278/350
4/4 [==============================] - 0s 3ms/step - loss: 1591687.0250
Epoch 279/350
4/4 [==============================] - 0s 3ms/step - loss: 1585654.7000
Epoch 280/350
4/4 [==============================] - 0s 3ms/step - loss: 1614316.3250
Epoch 281/350
4/4 [==============================] - 0s 3ms/step - loss: 1488266.9250
Epoch 282/350
4/4 [==============================] - 0s 3ms/step - loss: 1495857.9750
Epoch 283/350
4/4 [==============================] - 0s 4ms/step - loss: 1424749.8250
Epoch 284/350
4/4 [==============================] - 0s 3ms/step - l

In [203]:
def lstm_model(x_train, y_train, x_test, y_test, model):
    err = 0
    result = []
    for i,j in zip(x_test, y_test):
        i = i.reshape(1, 8, 1)
        print(i*10)
        predict = model.predict(i)
        print(predict)
        predict = predict[0][0]
        j = j[0]
        err = (predict-j)**2
        result.append(err)
        print(math.sqrt(err))
    return math.sqrt(sum(result)/len(result))
print(lstm_model(x_train, y_train, x_test, y_test, regressor))

[[[29110.]
  [26450.]
  [30020.]
  [28240.]
  [29070.]
  [28850.]
  [29750.]
  [31390.]]]
[[[2651.2483]
  [2651.2483]
  [2651.2483]
  [2651.2483]
  [2651.2483]
  [2651.2483]
  [2651.248 ]
  [2651.248 ]]]
337.75171169070336
[[[26450.]
  [30020.]
  [28240.]
  [29070.]
  [28850.]
  [29750.]
  [31390.]
  [29890.]]]
[[[2651.2483]
  [2651.2483]
  [2651.2483]
  [2651.2483]
  [2651.2483]
  [2651.2483]
  [2651.248 ]
  [2651.248 ]]]
457.7517067144589
[[[30020.]
  [28240.]
  [29070.]
  [28850.]
  [29750.]
  [31390.]
  [29890.]
  [31090.]]]
[[[2651.2483]
  [2651.2483]
  [2651.2483]
  [2651.2483]
  [2651.2483]
  [2651.2483]
  [2651.248 ]
  [2651.248 ]]]
397.7517088272532
[[[28240.]
  [29070.]
  [28850.]
  [29750.]
  [31390.]
  [29890.]
  [31090.]
  [30490.]]]
[[[2651.2483]
  [2651.2483]
  [2651.2483]
  [2651.2483]
  [2651.2483]
  [2651.2483]
  [2651.248 ]
  [2651.248 ]]]
377.75171657452466
[[[29070.]
  [28850.]
  [29750.]
  [31390.]
  [29890.]
  [31090.]
  [30490.]
  [30290.]]]
[[[2651.2483]
  [265

In [182]:
submit = pd.DataFrame()
submit['date'] = ['20210323', '20210324', '20210325', '20210326', '20210327', '20210328', '20210329']
predict_list = predict_mean(4)
submit['operating_reserve(MW)'] = predict_list
submit

AttributeError: module 'numpy' has no attribute 'de'

In [244]:
def baseline_model(shape):
    # create model
    model = Sequential()
    model.add(Dense(13, kernel_initializer='normal', input_shape=(shape[1], shape[2]), activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [491]:
data = pd.read_csv('data.csv')

def create_week_data(past = 3):
    # model = build()
    x_train = []
    y_train = []
    x_test = []
    y_test = []
    for i in range(0, len(data)-7*past+7):
        t = []
        for j in range(past):
            t.append(data.iloc[i+j*7][['備轉']])
            try:
                y = data['備轉'].iloc[i+j*7+7]
            except:
                y = None
        if i >= 2586 and i <= 2613:
            continue
#         if i < 2600:
#             x_train.append(t)
#             y_train.append(y)
#         else:
#             x_test.append(t)
#             y_test.append(y)
        y_test.append(y)
        x_test.append(x)

    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_test = np.array(x_test)
    y_test = np.array(y_test)
    return x_train, y_train, x_test, y_test
x_train, y_train, x_test, y_test = create_week_data(3)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_test[-1])

(0,)
(0,)
(2594, 1, 7)
(2594,)
[[3087.  3183.  3214.  3137.  2839.  3186.  2960.7]]


In [471]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split



In [481]:
past = 3
x_train, y_train, x_test, y_test = create_week_data(past)


model = baseline_model(x_test.shape)
x_train = x_train.reshape(-1, past)
x_test = x_test.reshape(-1, past)
print(x_test.shape)
regr = MLPRegressor(random_state=1, max_iter=2000, hidden_layer_sizes=10).fit(x_train, y_train)

# x_train, y_train, x_test, y_test = create_week_data(3)


# model = baseline_model(x_test.shape)
# x_train = x_train.reshape(-1, 3)
# x_test = x_test.reshape(-1, 3)
# print(x_test.shape)
# regr = MLPRegressor(random_state=1, max_iter=1000, hidden_layer_sizes=10).fit(x_train, y_train)

(87, 3)


In [486]:
predict = []
for i in range(7):
    x = np.array([3087,3183, 3214, 3137, 2839, 3186, 2960.7]).reshape(-1, 7)
    print(x)
    print(regr.predict(x))
    predict.append(regr.predict(x)[0])
print(predict)

[[3087.  3183.  3214.  3137.  2839.  3186.  2960.7]]


ValueError: X has 7 features, but MLPRegressor is expecting 3 features as input.

In [469]:
# predict = [3168, 3154, 3176, 3275, 2906, 2957, 3136]
# result = [316,321,322,316,301,279,314]
# result = [i*10 for i in result]
result = [2933, 3074, 3087, 3183, 3214, 3137, 2839]
err = 0
for i in range(7):
    err += (result[i]-predict[i])**2
print(math.sqrt(err/7))

191.4234236396863
